In [0]:
#!pip install -U -q PyDrive

#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)


In [2]:
# Library loading
%matplotlib inline

import pandas as pd # manipulate dataframes
import matplotlib.pyplot as plt # plotting
import numpy as np

import matplotlib, time, h5py, torch, neuravi

from sklearn.metrics import mean_squared_error

# First we check if CUDA is available
print("CUDA AVAILABLE? ",torch.cuda.is_available())

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
      
device = get_default_device()
print(device)

CUDA AVAILABLE?  True
cuda


# General function

For this one we use custom training functions (as we will change the objective function)

In [39]:
def training(neuralmodel,ds,criterion,optimizer,save_name,train_patience = 50,verbose=True, mode="main"):
    if verbose == True:
        time1 = time.time()
        
        if mode == "pretrain":
            print("! Pretrain mode...\n")
        else:
            print("Full training.\n")

    neuralmodel.train()

    # for early stopping
    epoch = 0
    best_epoch = 0
    val_ex = 0

    # for recording losses
    record_train_loss = []
    record_valid_loss = []

    while val_ex <= train_patience:
        optimizer.zero_grad()

        # Forward pass
        y_ag_pred_train = neuralmodel.ag(ds.x_visco_train,ds.T_visco_train)
        y_myega_pred_train = neuralmodel.myega(ds.x_visco_train,ds.T_visco_train)
        y_am_pred_train = neuralmodel.am(ds.x_visco_train,ds.T_visco_train)
        y_cg_pred_train = neuralmodel.cg(ds.x_visco_train,ds.T_visco_train)
        y_raman_pred_train = neuralmodel.raman_pred(ds.x_raman_train)
        y_density_pred_train = neuralmodel.density(ds.x_density_train)
        y_entro_pred_train = neuralmodel.sctg(ds.x_entro_train)
        y_tg_pred_train = neuralmodel.tg(ds.x_tg_train)
        y_cp_pred_train = neuralmodel.dCp(ds.x_entro_train,neuralmodel.tg(ds.x_entro_train))
        y_ri_pred_train = neuralmodel.sellmeier(ds.x_ri_train, ds.lbd_ri_train)
        
        # on validation set
        y_ag_pred_valid = neuralmodel.ag(ds.x_visco_valid,ds.T_visco_valid)
        y_myega_pred_valid = neuralmodel.myega(ds.x_visco_valid,ds.T_visco_valid)
        y_am_pred_valid = neuralmodel.am(ds.x_visco_valid,ds.T_visco_valid)
        y_cg_pred_valid = neuralmodel.cg(ds.x_visco_valid,ds.T_visco_valid)
        y_raman_pred_valid = neuralmodel.raman_pred(ds.x_raman_valid)
        y_density_pred_valid = neuralmodel.density(ds.x_density_valid)
        y_entro_pred_valid = neuralmodel.sctg(ds.x_entro_valid)
        y_tg_pred_valid = neuralmodel.tg(ds.x_tg_valid)
        y_cp_pred_valid = neuralmodel.dCp(ds.x_entro_valid,neuralmodel.tg(ds.x_entro_valid))
        y_ri_pred_valid = neuralmodel.sellmeier(ds.x_ri_valid, ds.lbd_ri_valid)
        
        # Compute Loss

        # train
        loss_ag = criterion(y_ag_pred_train, ds.y_visco_train)
        loss_myega = criterion(y_myega_pred_train, ds.y_visco_train)
        loss_am = criterion(y_am_pred_train, ds.y_visco_train)
        loss_cg = criterion(y_cg_pred_train, ds.y_visco_train)
        loss_raman = criterion(y_raman_pred_train,ds.y_raman_train)
        loss_tg = criterion(y_tg_pred_train,ds.y_tg_train)
        loss_density = criterion(y_density_pred_train,ds.y_density_train)
        loss_entro = criterion(y_entro_pred_train,ds.y_entro_train)
        loss_ri = criterion(y_ri_pred_train,ds.y_ri_train)
        
        if mode == "pretrain":
            loss = 0.001*loss_tg #+ 10*loss_raman #+ 1000*loss_density #+ loss_entro #+ loss_ri*1000
        else:
            loss = loss_ag + loss_myega + loss_am + loss_cg #+ 10*loss_raman #+ 1000*loss_density #+ loss_entro #+ loss_ri*1000

        # validation
        loss_ag_v = criterion(y_ag_pred_valid, ds.y_visco_valid)
        loss_myega_v = criterion(y_myega_pred_valid, ds.y_visco_valid)
        loss_am_v = criterion(y_am_pred_valid, ds.y_visco_valid)
        loss_cg_v = criterion(y_cg_pred_valid, ds.y_visco_valid)
        loss_raman_v = criterion(y_raman_pred_valid,ds.y_raman_valid)
        loss_tg_v = criterion(y_tg_pred_valid,ds.y_tg_valid)
        loss_density_v = criterion(y_density_pred_valid,ds.y_density_valid)
        loss_entro_v = criterion(y_entro_pred_valid,ds.y_entro_valid)
        loss_ri_v = criterion(y_ri_pred_valid,ds.y_ri_valid)

        if mode == "pretrain":
            loss_v = 0.001*loss_tg_v #+ 10*loss_raman_v #+ 1000*loss_density_v #+ loss_entro_v #+ loss_ri_v*1000
        else:
            loss_v = loss_ag_v + loss_myega_v + loss_am_v + loss_cg_v #+ 10*loss_raman_v #+ 1000*loss_density_v #+ loss_entro_v #+ loss_ri*1000

        record_train_loss.append(loss.item())
        record_valid_loss.append(loss_v.item())

        if verbose == True:
            if (epoch % 200 == 0):
              print('Epoch {} => train loss: {}; valid loss: {}'.format(epoch, loss.item(), loss_v.item()))

        # calculating ES criterion
        if epoch == 0:
            val_ex = 0
            best_loss_v = loss_v.item()
        elif loss_v.item() <= best_loss_v:
            val_ex = 0
            best_epoch = epoch
            best_loss_v = loss_v.item()

            # save best model
            torch.save(neuralmodel.state_dict(), save_name)
        else:
            val_ex += 1

        # Backward pass
        loss.backward()
        optimizer.step()

        epoch += 1

    if verbose == True:
        time2 = time.time()
        print("Running time in seconds:", time2-time1)

    return neuralmodel, record_train_loss, record_valid_loss

In [40]:
# Load dataset
ds = neuravi.data_loader("./data/DataSet_0p20val.hdf5",
                         "./data/NKAS_Raman.hdf5",
                         "./data/NKAS_density.hdf5",
                         "./data/NKAS_optical.hdf5",device)


In [41]:
nb_exp = 10

for i in range(nb_exp):
    print('Experiment {} started...'.format(i))
    
    path_out = "./model/exp_multitask/loss_ag_myega_am_cg_"+str(i)+".pth"
    
    # Network declaration
    neuralmodel = neuravi.model(4,200,3,ds.nb_channels_raman,p_drop=0.05) 

    # criterion for match
    criterion = torch.nn.MSELoss()
    criterion.to(device)
    optimizer = torch.optim.Adam(neuralmodel.parameters(), lr = 0.001) # optimizer

    # we initialize the output bias
    neuralmodel.output_bias_init()

    # we send the neural net on device
    neuralmodel.to(device)
    
    neuralmodel, record_pretrain_loss, record_prevalid_loss = training(neuralmodel,ds,criterion,optimizer,path_out,mode="pretrain")
    neuralmodel, record_train_loss, record_valid_loss = training(neuralmodel,
                                                                         ds,
                                                                         criterion,optimizer,
                                                                         path_out,verbose=True,
                                                                         train_patience=25)

Experiment 0 started...
! Pretrain mode...

Epoch 0 => train loss: 46.19532012939453; valid loss: 61.42976379394531
Epoch 200 => train loss: 0.8780887722969055; valid loss: 1.2617266178131104
Running time in seconds: 6.158773183822632
Full training.

Epoch 0 => train loss: 5.872018814086914; valid loss: 7.443633079528809
Running time in seconds: 8.3176109790802
Experiment 1 started...
! Pretrain mode...

Epoch 0 => train loss: 46.76251220703125; valid loss: 63.8747444152832
Epoch 200 => train loss: 0.9418908953666687; valid loss: 1.3002619743347168
Running time in seconds: 7.472977876663208
Full training.

Epoch 0 => train loss: 5.349086284637451; valid loss: 6.946389198303223
Running time in seconds: 4.361438751220703
Experiment 2 started...
! Pretrain mode...

Epoch 0 => train loss: 49.54499816894531; valid loss: 54.52342987060547
Epoch 200 => train loss: 1.0473253726959229; valid loss: 1.5573610067367554
Epoch 400 => train loss: 0.4805125892162323; valid loss: 0.7145112752914429
Run

In [0]:
p_drop


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])